In [1]:
%cd ..

C:\Users\georg\Documents\msc-projcet


In [2]:
import pandas as pd
from src.data.emails import clean_content,compute_sentiment
from textblob import TextBlob
import nltk
from datetime import date

In [3]:
# nltk.download('punkt')

In [4]:
wg = "calext"
ml = "calsify"
docs_df = pd.read_pickle(f"data/raw/{wg}/documents.pkl")
events_df = pd.read_pickle(f"data/raw/{wg}/events.pkl")
emails_df = pd.read_pickle(f"data/raw/{ml}/emails.pkl")

print(f"Loaded {docs_df.shape[0]} documents, {events_df.shape[0]} events, and {emails_df.shape[0]} emails")

Loaded 20 documents, 1293 events, and 3048 emails


In [5]:
emails_df['content'] = emails_df['content'].apply(clean_content)
emails_df['sentiment'] = emails_df.apply(compute_sentiment,axis=1)
events_df['timestamp'] = events_df['timestamp'].apply(pd.to_datetime)

In [6]:
emails_df = emails_df[emails_df['sentiment'].notnull()]

In [13]:
filter_revisions = (events_df['type'] == 'new_revision') | (events_df['type'] == 'rfc_published')
grouped_events = events_df[filter_revisions].groupby('doc_id')

In [39]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("I think this is not correct")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [13]:
sample = emails_df.sample(10)
sample

,timestamp,from,subject,content,charset,sentiment
2978,2021-02-24 23:00:02,Murray Kucherawy via Datatracker <noreply@ietf...,[calsify] Murray Kucherawy's No Objection on d...,b'Murray Kucherawy has entered the following b...,utf-8,0.250287
289,2005-04-15 01:32:18,"""reinhold at kainhofer.com""",[Ietf-calsify] PERIOD types,b' But we agree that you have to have the time...,None,0.050200
17,2004-08-17 12:45:53,"""lisa at osafoundation.org""",[Ietf-calsify] Just for the record. Notes has ...,"b""I think Cameron may be right, even though I'...",None,0.090300
319,2005-08-08 22:28:24,"""lisa at osafoundation.org""",[Ietf-calsify] Timezone registration,b'That\'s not a bad scheme but unnecessary wit...,None,0.316243
2669,2020-07-13 11:31:22,internet-drafts@ietf.org,[calsify] I-D Action: draft-ietf-calext-valarm...,b'A New Internet-Draft is available from the o...,utf-8,0.104100
1776,2017-02-10 22:56:08,Michael Douglass <mikeadouglass@gmail.com>,Re: [calsify] Review of draft-ietf-calext-ical...,"b""Thanks Philipp. I'll resubmit with some of t...",windows-1252,0.122187
342,2005-09-02 09:11:24,"""Carsten.Guenther at scalix.com""",[Ietf-calsify] DTEND for day events,"b""Unfortunately it's not just iCal. Just to na...",None,0.047575
2001,2018-08-13 18:09:08,Robert Stepanek <rsto@fastmailteam.com>,[calsify] Requesting JSCalendar WGLC,"b""About a month ago we updated the JSCalendar ...",utf-8,0.208750
563,2006-10-11 08:37:23,"""cyrus at daboo.name""",[Ietf-calsify] summary of jabber sessions and ...,"b'Hi Reinhold, -- An instance representing the...",None,-0.034175
1557,2015-02-05 11:16:30,Cyrus Daboo <cyrus@daboo.name>,Re: [calsify] SKIP was Re: AD review of draft-...,"b'Hi Barry, -- I am not convinced that there i...",us-ascii,0.045571


In [17]:
sample.loc[319]['content']

b'That\'s not a bad scheme but unnecessary with iCalendar. In iCalendar you can define a timezone that is -0800 from 1987 until 2007 and then something else from 2007 onward. In fact it\'s necessary to put both the pre and post-2007 rules into one timezone because a meeting that recurs over a time period that crosses from 2006 into 2007 needs to have its recurrences calculated out according to both the old and the new DST rules. I quite liked what I heard from Patrik at the meeting, that people actually schedule meetings with reference to the time in a particular *place* rather than in a particular timezone. I\'ve seen more software recently, probably based on the Olsen database, allow the user to pick a place like "America/San Francisco" rather than a timezone like PST. I hope that means is that when I schedule recurring meetings in San Francisco the recurrence is always calculated according to the best known time in San Francisco even if San Francisco were to secede from PST. Lisa '